# Import libraries

In [1]:
import os
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import random
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix
from PIL import Image

tfk = tf.keras
tfkl = tf.keras.layers
print(tf.__version__)
device_name = tf.test.gpu_device_name()
if "GPU" not in device_name:
    print("GPU device not found")
print('Found GPU at: {}'.format(device_name))

2.6.4
Found GPU at: /device:GPU:0


2022-11-14 10:52:01.918979: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-14 10:52:01.970578: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-14 10:52:02.047109: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-14 10:52:02.047914: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA 

In [2]:
# Random seed for reproducibility
seed = 42

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

In [3]:
!ls /kaggle/input/mobilenet-epoch10/models/test

keras_metadata.pb  saved_model.pb  variables


In [4]:
print("--> Checking for physical Tensorflow devices")
for device in tf.config.list_physical_devices():
    print(": {}".format(device.name))

--> Checking for physical Tensorflow devices
: /physical_device:CPU:0
: /physical_device:GPU:0


2022-11-14 10:52:04.845366: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-14 10:52:04.846264: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-14 10:52:04.847047: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


# Load data

Solo questo dovrebbe essere diverso durante la challenge

In [5]:
dataset_dir = '/kaggle/input/homework1/training_data_final'
image_size = (96,96)

# Dataloader and Data Augmentation

We can fine tune this

In [6]:
noaug_train_data_gen = tfk.preprocessing.image.ImageDataGenerator(
                                        rescale=1/255.)

aug_train_data_gen = tfk.preprocessing.image.ImageDataGenerator(rotation_range=30,
                                        height_shift_range=50,
                                        width_shift_range=50,
                                        zoom_range=0.3,
                                        horizontal_flip=True,
                                        vertical_flip=True, 
                                        fill_mode='reflect',
                                        rescale=1/255.,
                                        validation_split=0.2)

aug_validation_data_gen = tfk.preprocessing.image.ImageDataGenerator(
                                        rescale=1/255.,
                                        validation_split=0.2)

train_gen = aug_train_data_gen.flow_from_directory(directory=dataset_dir,
                                               target_size=image_size,
                                               color_mode='rgb',
                                               class_mode='categorical',
                                               batch_size=8,
                                               shuffle=True,
                                               seed=seed,
                                               subset='training')

train_gen_all = noaug_train_data_gen.flow_from_directory(directory=dataset_dir,
                                               target_size=image_size,
                                               color_mode='rgb',
                                               class_mode='categorical',
                                               batch_size=8,
                                               seed=seed)

validation_gen = aug_validation_data_gen.flow_from_directory(directory=dataset_dir,
                                               target_size=image_size,
                                               color_mode='rgb',
                                               class_mode='categorical',
                                               batch_size=8,
                                               shuffle=True,
                                               seed=seed,
                                               subset='validation')

Found 2836 images belonging to 8 classes.
Found 3542 images belonging to 8 classes.
Found 706 images belonging to 8 classes.


In [7]:
print("Assigned labels")
print(train_gen.class_indices)
print()
print("Target classes")
print(train_gen.classes)
num_classes = train_gen.num_classes

Assigned labels
{'Species1': 0, 'Species2': 1, 'Species3': 2, 'Species4': 3, 'Species5': 4, 'Species6': 5, 'Species7': 6, 'Species8': 7}

Target classes
[0 0 0 ... 7 7 7]


# Load pre-trained model for transfer learning

We can fine tune this with different pretrained models

In [8]:
# Create the base model from the pre-trained model
IMG_SHAPE = image_size + (3,)
base_model = tf.keras.applications.EfficientNetB2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

2022-11-14 10:52:05.545335: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-14 10:52:05.546237: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-14 10:52:05.546968: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-14 10:52:05.547762: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-14 10:52:05.548426: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

31801344/31790344 [==============================] - 2s 0us/step


In [9]:
base_model.summary()

Model: "efficientnetb2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 96, 96, 3)]  0                                            
__________________________________________________________________________________________________
rescaling (Rescaling)           (None, 96, 96, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
normalization (Normalization)   (None, 96, 96, 3)    7           rescaling[0][0]                  
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 97, 97, 3)    0           normalization[0][0]              
_____________________________________________________________________________________

In [10]:
def build_model(input_shape, output_classes, learning_rate=0.001, freeze=True):

    # Build the neural network layer by layer
    input_layer = tfkl.Input(shape=input_shape, name='input_layer')

    # Freeze the base model
    if(freeze):
      base_model.trainable = False
    else:
      base_model.trainable = True
    # We need training=False for the BatchNormalization layer
    feature_extractor = base_model(input_layer , training=not freeze)

    x = tfkl.GlobalAveragePooling2D()(feature_extractor)
    x = tfkl.Dropout(0.1)(x)
    x = tfkl.Dense(units=256, activation='relu', kernel_initializer=tfk.initializers.GlorotUniform(seed), name='hidden_layer_1')(x)
    x = tfkl.Dropout(0.2)(x)
    output_layer = tfkl.Dense(units=output_classes, activation='softmax', kernel_initializer=tfk.initializers.GlorotUniform(seed), name='output_layer')(x)

    # Connect input and output through the Model class
    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='model')

    # Compile the model
    model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(learning_rate=learning_rate), metrics='accuracy')

    # Return the model
    return model

def build_custom_model(input_shape, output_classes, learning_rate=0.01):
    chanDim = -1
    levels = 4
    input_layer = tfkl.Input(shape=input_shape, name='input_layer')
    x = tfkl.Conv2D(32, (7, 7), padding="same")(input_layer)
    x = tfkl.LeakyReLU(alpha=0.3)(x)
    x = tfkl.BatchNormalization(axis=chanDim)(x)
    x = tfkl.Add([input_layer,x])
    # CNN 
    for i in range(levels):
        first = tfkl.MaxPooling2D(pool_size=(3, 3))(x)
        x = tfkl.Conv2D(32*(i+2), (3, 3), padding="same")(first)
        x = tfkl.LeakyReLU(alpha=0.3)(x)
        x = tfkl.BatchNormalization(axis=chanDim)(x)
        x = tfkl.Add([first,x])
    x = tfkl.MaxPooling2D(pool_size=(3, 3))(x)
    
    # FC
    x = tfkl.GlobalAveragePooling2D()(x)
    x = tfkl.Dropout(0.2)(x)
    x = tfkl.Dense(units=256, activation='relu', kernel_initializer=tfk.initializers.GlorotUniform(seed), name='hidden_layer_1')(x)
    x = tfkl.Dropout(0.3)(x)
    output_layer = tfkl.Dense(units=output_classes, activation='softmax', kernel_initializer=tfk.initializers.GlorotUniform(seed), name='output_layer')(x)
    
    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='model')
              
    model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(learning_rate=learning_rate),metrics="accuracy")
    return model

In [11]:
saved = True
if(saved):
    model = tfk.models.load_model('/kaggle/input/efficientnetb2-200epochs/EfficientNetB2_200epoch.h5')   
else:
    model = build_custom_model(IMG_SHAPE, num_classes)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (InputLayer)     [(None, 96, 96, 3)]       0         
_________________________________________________________________
efficientnetb2 (Functional)  (None, 3, 3, 1408)        7768569   
_________________________________________________________________
global_average_pooling2d_1 ( (None, 1408)              0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 1408)              0         
_________________________________________________________________
hidden_layer_1 (Dense)       (None, 256)               360704    
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
_________________________________________________________________
output_layer (Dense)         (None, 8)                 2056  

In [12]:
train = False
if(train):
    history = model.fit(
            x = train_gen,
            epochs = 100,
            validation_data = validation_gen
        ).history
    model.save("./models/EfficientNetB2_200epoch.h5")


In [13]:
!ls models

ls: cannot access 'models': No such file or directory


In [14]:
# Retrain with all dataset with small learning rate
last_train = True
if(last_train):
    history = model.fit(
                x = train_gen_all,
                epochs = 100
            ).history
    model.save("./models/EfficientNetB2_final.h5")

2022-11-14 10:52:17.825320: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/100


2022-11-14 10:52:26.315439: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


443/443 [==============================] - 44s 67ms/step - loss: 0.8016 - accuracy: 0.7352
Epoch 2/100
443/443 [==============================] - 29s 65ms/step - loss: 0.7482 - accuracy: 0.7558
Epoch 3/100
443/443 [==============================] - 29s 65ms/step - loss: 0.7827 - accuracy: 0.7422
Epoch 4/100
443/443 [==============================] - 29s 66ms/step - loss: 0.7090 - accuracy: 0.7643
Epoch 5/100
443/443 [==============================] - 29s 65ms/step - loss: 0.6846 - accuracy: 0.7724
Epoch 6/100
443/443 [==============================] - 29s 66ms/step - loss: 0.6856 - accuracy: 0.7770
Epoch 7/100
443/443 [==============================] - 30s 67ms/step - loss: 0.6636 - accuracy: 0.7767
Epoch 8/100
443/443 [==============================] - 29s 66ms/step - loss: 0.7005 - accuracy: 0.7772
Epoch 9/100
443/443 [==============================] - 29s 65ms/step - loss: 0.6294 - accuracy: 0.8027
Epoch 10/100
443/443 [==============================] - 30s 67ms/step - loss: 0.6587 

/opt/conda/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)
